In [1]:
# Importing Pandas for Data Operations
import pandas as pd
import os
os.chdir("/home/lokeshstat/Downloads")
df=pd.read_csv('data.csv')

In [2]:
#Printing Head of Data Set 
df.head()

,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
0,P1,Persistent,Male,Caucasian,Not Hispanic,West,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
1,P2,Non-Persistent,Male,Asian,Not Hispanic,West,55-65,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
2,P3,Non-Persistent,Female,Other/Unknown,Hispanic,Midwest,65-75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,Y,N,N,N,N,N,N,N,2
3,P4,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1
4,P5,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1


In [3]:
#Calculation of Missing Values
df_missing = df.isna()
df_missing = df_missing.sum()
print(df_missing)
df_missing=pd.DataFrame(df_missing)
df_missing.to_csv('missing.csv')

Ptid                              0
Persistency_Flag                  0
Gender                            0
Race                              0
Ethnicity                         0
                                 ..
Risk_Hysterectomy_Oophorectomy    0
Risk_Estrogen_Deficiency          0
Risk_Immobilization               0
Risk_Recurring_Falls              0
Count_Of_Risks                    0
Length: 69, dtype: int64


In [4]:
#Freq Distribution of Cat Variables
cat_list=[x for x in df.columns if x not in ['Ptid',
'Persistency_Flag',
'Count_Of_Risks',
'Dexa_Freq_During_Rx',
]]

freq_list=[]
percentage_list=[]
var_list=[]
header=[]

for i in cat_list:
    cat_dict=dict(df[i].value_counts())   


    for var_cat in cat_dict.keys():
        var_list.append(var_cat)

    for frequency in cat_dict.values():  
        freq_list.append(frequency)  
        percentage_list.append(frequency/sum(cat_dict.values()))
        header.append(i) 
        
   
cat_freq =           {'Variable'  : header,
                       'Category'  : var_list,
                       'Frequency' : freq_list,
                       'Percentage': percentage_list}


freq = pd.DataFrame(cat_freq, columns =['Variable','Category','Frequency','Percentage'])


freq.to_csv( 'cat_freq.csv',  index = False)        
print(freq)

                     Variable          Category  Frequency  Percentage
0                      Gender            Female       3230    0.943341
1                      Gender              Male        194    0.056659
2                        Race         Caucasian       3148    0.919393
3                        Race     Other/Unknown         97    0.028329
4                        Race  African American         95    0.027745
..                        ...               ...        ...         ...
174  Risk_Estrogen_Deficiency                 Y         11    0.003213
175       Risk_Immobilization                 N       3410    0.995911
176       Risk_Immobilization                 Y         14    0.004089
177      Risk_Recurring_Falls                 N       3355    0.979848
178      Risk_Recurring_Falls                 Y         69    0.020152

[179 rows x 4 columns]


In [5]:

drop_list_1=['Ntm_Speciality',
'Ntm_Specialist_Flag',
'Risk_Osteogenesis_Imperfecta',
'Risk_Untreated_Chronic_Hyperthyroidism']

df = df.drop(columns = drop_list_1,axis=1)

In [6]:
# Target Mapping
mapping_target = {'Persistent':1, 'Non-Persistent':0}
df = df.replace({'Persistency_Flag':mapping_target})



#Creating intraction Variables
df['Gender_Age_Bucket_int']=df['Gender'].astype(str)+"_"+df['Age_Bucket'].astype(str)
df['Race_Ethnicity_int']=df['Race'].astype(str)+"_"+df['Ethnicity'].astype(str)


In [7]:
#List of Columns where One Hot Encoding is  required
ohe_list_1=['Gender',
'Age_Bucket',
'Race',
'Region',
'Ethnicity',
'Ntm_Speciality_Bucket',
'Risk_Segment_Prior_Ntm',
'Risk_Segment_During_Rx',
'Change_Risk_Segment',
'Tscore_Bucket_Prior_Ntm',
'Tscore_Bucket_During_Rx',
'Change_T_Score',
'Gender_Age_Bucket_int',
'Race_Ethnicity_int']


#one Hot Encoding of Cat columns
def one_hot_encoder(df, nan_as_category = False):
    original_columns = list(df.columns)
    categorical_columns = ohe_list_1
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

df_1, newCols = one_hot_encoder(df, nan_as_category=False)

In [8]:
#Flag Variable creation for dichotomous variables
not_in_binary=['Gender',
'Age_Bucket',
'Race',
'Region',
'Ethnicity',
'Ntm_Speciality_Bucket',
'Risk_Segment_Prior_Ntm',
'Risk_Segment_During_Rx',
'Change_Risk_Segment',
'Tscore_Bucket_Prior_Ntm',
'Tscore_Bucket_During_Rx',
'Change_T_Score',
'Ptid',
'Persistency_Flag',
'Count_Of_Risks',
'Dexa_Freq_During_Rx',
'Ntm_Speciality',
'Ntm_Specialist_Flag',
'Risk_Osteogenesis_Imperfecta',
'Risk_Untreated_Chronic_Hyperthyroidism',
'Gender_Age_Bucket_int',
'Race_Ethnicity_int']

binary_conversion=[x for x in df.columns if x not in not_in_binary]


bin_cnv_y_n= {'Y':1, 'N':0}
bin_conv_list=[x for x in binary_conversion if x not in ['Adherent_Flag']]

for i in bin_conv_list:
    df_1 = df_1.replace({i:bin_cnv_y_n})

    
bin_cnv_Adherent_Flag= {'Adherent':1, 'Non-Adherent':0}
df_1 = df_1.replace({'Adherent_Flag':bin_cnv_Adherent_Flag})    

In [9]:
#Removing variables having other/Unkown or Unknown Categoury and variables having more than 99.9% unique Values
drop_list_2=['Race_Other/Unknown',
'Region_Other/Unknown',
'Ethnicity_Unknown',
'Risk_Segment_During_Rx_Unknown',
'Change_Risk_Segment_Unknown',
'Tscore_Bucket_During_Rx_Unknown',
'Change_T_Score_Unknown',
'Race_Ethnicity_int_African American_Hispanic',
'Race_Ethnicity_int_Asian_Hispanic',             
'Race_Ethnicity_int_African American_Unknown',
'Race_Ethnicity_int_Asian_Unknown',
'Race_Ethnicity_int_Caucasian_Unknown',
'Race_Ethnicity_int_Other/Unknown_Hispanic',
'Race_Ethnicity_int_Other/Unknown_Not Hispanic',
'Race_Ethnicity_int_Other/Unknown_Unknown']

df_1 = df_1.drop(columns = drop_list_2,axis=1)

In [10]:
#Defining Training columns and target 
target = 'Persistency_Flag'
no_train=['Ptid','Persistency_Flag']
trainOn = [x for x in df_1.columns if x not in no_train]

In [11]:
#Removing the column having more than 99.9% unique values
toDrop = [x for x in df_1.columns if (max(df_1[x].value_counts()/df_1.shape[0])>0.999)]
df_1 = df_1.drop(columns = toDrop)

In [12]:
numerical_list=['Dexa_Freq_During_Rx','Count_Of_Comorbidity','Count_Of_Concomitant_Drug','Count_Of_Risks']
import numpy as np 
import scipy.stats as ss 
#Defining Cramer's Statistics
def cramers_stat(confusion_matrix): 
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum() 
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1))) 

cramers_list=[x for x in trainOn if x not in numerical_list]

#Calculating Cramer's Statistics between Target and Indipendent Variables
temp=pd.DataFrame([])
for col in cramers_list:
    result = cramers_stat(pd.crosstab(df_1[col],df_1[target]))[0:1]
    temp=pd.concat([temp,pd.concat([pd.DataFrame([col]), pd.DataFrame([result])],axis=1)],axis=0)

temp.columns = ['var', 'cramer']    
print(temp)
temp.to_csv('Cramer_V_DV_IV.csv')

                                                 var    cramer
0                             Gluco_Record_Prior_Ntm  0.006392
0                             Gluco_Record_During_Rx  0.268500
0                                     Dexa_During_Rx  0.621984
0                                Frag_Frac_Prior_Ntm  0.000635
0                                Frag_Frac_During_Rx  0.134255
..                                               ...       ...
0                     Gender_Age_Bucket_int_Male_>75  0.000125
0   Race_Ethnicity_int_African American_Not Hispanic  0.020907
0              Race_Ethnicity_int_Asian_Not Hispanic  0.048084
0              Race_Ethnicity_int_Caucasian_Hispanic  0.012146
0          Race_Ethnicity_int_Caucasian_Not Hispanic  0.005841

[93 rows x 2 columns]


In [13]:
#Selecting Variables where Cramers > 0.2 from the previous step
cramers_list_2=['Dexa_During_Rx',
'Comorb_Long_Term_Current_Drug_Therapy',
'Comorb_Encounter_For_Screening_For_Malignant_Neoplasms',
'Comorb_Encounter_For_Immunization',
'Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx',
'Concom_Viral_Vaccines',
'Concom_Systemic_Corticosteroids_Plain',
'Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified',
'Comorb_Gastro_esophageal_reflux_disease',
'Comorb_Personal_History_Of_Other_Diseases_And_Conditions',
'Comorb_Other_Disorders_Of_Bone_Density_And_Structure',
'Concom_Broad_Spectrum_Penicillins',
'Concom_Macrolides_And_Similar_Types',
'Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx',
'Concom_Anaesthetics_General',
'Concom_Cephalosporins',
'Ntm_Speciality_Bucket_Endo/Onc/Uro',
'Comorb_Dorsalgia',
'Concom_Narcotics',
'Concom_Fluoroquinolones',
'Comorb_Personal_history_of_malignant_neoplasm',
'Comorb_Vitamin_D_Deficiency',
'Ntm_Speciality_Bucket_OB/GYN/Others/PCP/Unknown',
'Gluco_Record_During_Rx']

#Calculating Cramers between indipendent Variables
temp=pd.DataFrame([])
for col1 in cramers_list_2:
    for col2 in cramers_list_2:
        if col1==col2:
            break
        else:
            result = cramers_stat(pd.crosstab(df_1[col1],df_1[col2]))[0:1]
            temp=pd.concat([temp,pd.concat([pd.DataFrame([col1]),pd.DataFrame([col2]), pd.DataFrame([result])],axis=1)],axis=0)
        
temp.columns = ['var1','var2', 'cramer'] 
print(temp)
temp.to_csv('Cramer_V_IV_IV.csv')

                                                 var1  \
0               Comorb_Long_Term_Current_Drug_Therapy   
0   Comorb_Encounter_For_Screening_For_Malignant_N...   
0   Comorb_Encounter_For_Screening_For_Malignant_N...   
0                   Comorb_Encounter_For_Immunization   
0                   Comorb_Encounter_For_Immunization   
..                                                ...   
0                              Gluco_Record_During_Rx   
0                              Gluco_Record_During_Rx   
0                              Gluco_Record_During_Rx   
0                              Gluco_Record_During_Rx   
0                              Gluco_Record_During_Rx   

                                               var2    cramer  
0                                    Dexa_During_Rx  0.279843  
0                                    Dexa_During_Rx  0.320680  
0             Comorb_Long_Term_Current_Drug_Therapy  0.166937  
0                                    Dexa_During_Rx  0.2583

In [14]:
#Dropping Variables having Correlation > =  0.7
drop_list=['Ntm_Speciality_Bucket_OB/GYN/Others/PCP/Unknown','Gluco_Record_During_Rx']
trainOn=[x for x in cramers_list_2 if x not in drop_list]

In [15]:
#Calculating Fisher Scores for Numerical Variables
num_list_fisher=['Dexa_Freq_During_Rx','Count_Of_Risks']
temp = pd.DataFrame()
for i in num_list_fisher:
    mean_nb = df_1.loc[df_1[target] == 0, i].mean()
    mean_b = df_1.loc[df_1[target]== 1, i].mean()
    var_nb = df_1.loc[df_1[target] == 0, i].var()
    var_b = df_1.loc[df_1[target] == 1, i].var()

    if (var_nb+var_b==0):
        continue
    fs = np.abs(mean_nb - mean_b)/np.sqrt(var_nb + var_b)
    temp = pd.concat([temp, pd.concat([pd.DataFrame([i]), pd.DataFrame([mean_nb]), pd.DataFrame([mean_b]), pd.DataFrame([var_nb]), pd.DataFrame([var_b]), pd.DataFrame([fs])], axis = 1)])

fisher_score = temp
fisher_score.columns = ['var', 'mean_nb', 'mean_b', 'var_nb', 'var_b', 'score']
fisher_score.to_csv('fisher_score.csv')

In [16]:
#Caculating Correlation Between Numerical Variables
temp=pd.DataFrame([])
for col1 in num_list_fisher:
    for col2 in num_list_fisher:
        if col1==col2:
            break
        else:
            result = np.corrcoef(df_1[col1],df_1[col2]).min()
            temp=pd.concat([temp,pd.concat([pd.DataFrame([col1]),pd.DataFrame([col2]), pd.DataFrame([result])],axis=1)],axis=0)

temp.columns = ['var1','var2', 'corr']
print(temp)
temp.to_csv('corr.csv')

             var1                 var2      corr
0  Count_Of_Risks  Dexa_Freq_During_Rx  0.013964


In [17]:
#Selecting final Variables
final_model_var=['Concom_Broad_Spectrum_Penicillins',
'Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified',
'Comorb_Encounter_For_Immunization',
'Comorb_Gastro_esophageal_reflux_disease',
'Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx',
'Concom_Viral_Vaccines',
'Comorb_Vitamin_D_Deficiency',
'Comorb_Encounter_For_Screening_For_Malignant_Neoplasms',
'Ntm_Speciality_Bucket_Endo/Onc/Uro',
'Comorb_Long_Term_Current_Drug_Therapy',
'Dexa_During_Rx']

In [18]:
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
from bayes_opt import BayesianOptimization
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc
from sklearn import metrics
from sklearn import preprocessing
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")
import itertools
from scipy import interp


train_df=df_1
features=final_model_var
target = 'Persistency_Flag'

#cut tr and val
bayesian_tr_idx, bayesian_val_idx = train_test_split(train_df[features], test_size = 0.2, random_state = 42, stratify = train_df[target])
bayesian_tr_idx = bayesian_tr_idx.index
bayesian_val_idx = bayesian_val_idx.index



#black box LGBM 
def LGB_bayesian(
    learning_rate,
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_samples, 
    min_child_weight,
    subsample, 
    min_data_in_leaf,
    max_depth,
    colsample_bytree,
    reg_alpha,
    reg_lambda,
    scale_pos_weight
     ):
    
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    param = {
              'num_leaves': num_leaves, 
              'min_child_samples': min_child_samples, 
              'min_data_in_leaf': min_data_in_leaf,
              'min_child_weight': min_child_weight,
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
              'subsample': subsample, 
              'max_depth': max_depth,
              'colsample_bytree': colsample_bytree,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight':scale_pos_weight,
              'objective': 'binary',
              'save_binary': True,
              'seed': 1337,
              'feature_fraction_seed': 1337,
              'bagging_seed': 1337,
              'drop_seed': 1337,
              'data_random_seed': 1337,
              'boosting_type': 'gbdt',
              'verbose': 1,
#              'is_unbalance': False,
              'boost_from_average': True,
              'metric':'auc'}    
    
    oof = np.zeros(len(train_df))
    categorical = []

    trn_data= lgb.Dataset(train_df.iloc[bayesian_tr_idx][features].values, label=train_df.iloc[bayesian_tr_idx][target].values)
    val_data= lgb.Dataset(train_df.iloc[bayesian_val_idx][features].values, label=train_df.iloc[bayesian_val_idx][target].values)

    clf = lgb.train(param, trn_data,  num_boost_round=100000, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 50)
    
    oof[bayesian_val_idx]  = clf.predict(train_df.iloc[bayesian_val_idx][features].values, num_iteration=clf.best_iteration)  
    
    score = roc_auc_score(train_df.iloc[bayesian_val_idx][target].values, oof[bayesian_val_idx])

    return score


# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (2, 50), 
    'min_data_in_leaf': (5, 150),
    'bagging_fraction' : (0.1,0.9),
    'feature_fraction' : (0.1,0.9),
    'learning_rate': (0.01, 0.07),
    'min_child_weight': (0.00001, 0.01),   
    'min_child_samples':(10, 500), 
    'subsample': (0.2, 0.8),
    'colsample_bytree': (0.4, 0.8), 
    'reg_alpha': (1, 10), 
    'reg_lambda': (1, 10),
    'max_depth':(-1,15),
    'scale_pos_weight':(1,100)
}




LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=42)



print(LGB_BO.space.keys)




init_points = 15
n_iter = 10




print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)




LGB_BO.max['target']
LGB_BO.max['params']

['bagging_fraction', 'colsample_bytree', 'feature_fraction', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_data_in_leaf', 'num_leaves', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample']
----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | baggin... | colsam... | featur... | learni... | max_depth | min_ch... | min_ch... | min_da... | num_le... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8513   |  0.3996   |  0.7803   |  0.6856   |  0.04592  |  1.496    |  86.44    |  0.000590 |  130.6    |  30.85    |  7.373    |  1.185    |  97.02    |  0.6995   |
|  2        |  0.8525   |  0.2699   |  0.4727   |  0.2467   |  0.02825  |  7.396    |  221.7    |

{'bagging_fraction': 0.1,
 'colsample_bytree': 0.7999999998367054,
 'feature_fraction': 0.1,
 'learning_rate': 0.06999999986765802,
 'max_depth': -1.0,
 'min_child_samples': 325.01757892868,
 'min_child_weight': 1e-05,
 'min_data_in_leaf': 5.0,
 'num_leaves': 50.0,
 'reg_alpha': 1.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 1.0,
 'subsample': 0.7999999995425457}

In [19]:
X = df_1[final_model_var]
y = df_1[target]

from sklearn.feature_selection import SelectFromModel
train_x, valid_x, train_y, valid_y = train_test_split(X[final_model_var], y, test_size=0.1, shuffle=True, stratify=y, random_state=9001)

from lightgbm import LGBMClassifier
lgbc=LGBMClassifier(n_estimators=10000,
                    bagging_fraction= 0.1,
                     colsample_bytree= 0.7999999998367054,
                     feature_fraction= 0.1,
                     learning_rate= 0.06999999986765802,
                     max_depth= -1,
                     min_child_samples= 325.01757892868,
                     min_child_weight= 1e-05,
                     min_data_in_leaf= 5,
                     num_leaves=50,
                     reg_alpha= 1.0,
                     reg_lambda= 1.0,
                     scale_pos_weight= 1,
                     subsample= 0.7999999995425457)

lgbc.fit(train_x, train_y, eval_set=[(train_x, train_y), 
                                     (valid_x, valid_y)],eval_metric= 'auc', verbose= 100, 
                                         early_stopping_rounds= 400,
                                         )
lgb_importance_df = pd.DataFrame()
lgb_importance_df["feature"] = final_model_var
lgb_importance_df["importance"] = lgbc.feature_importances_*(100)

lgbc.best_iteration_

lgb_importance_df.to_excel('fi_lgb.xlsx',index=False)


sub_preds = np.zeros(df.shape[0])
sub_preds = lgbc.predict_proba(df_1[final_model_var], num_iteration=lgbc.best_iteration_)[:, 1] 
df['score'] = sub_preds

Training until validation scores don't improve for 400 rounds
[100]	training's auc: 0.867908	training's binary_logloss: 0.438945	valid_1's auc: 0.844273	valid_1's binary_logloss: 0.458667
[200]	training's auc: 0.869678	training's binary_logloss: 0.430331	valid_1's auc: 0.84612	valid_1's binary_logloss: 0.453662
[300]	training's auc: 0.870218	training's binary_logloss: 0.428837	valid_1's auc: 0.846555	valid_1's binary_logloss: 0.452963
[400]	training's auc: 0.870612	training's binary_logloss: 0.428309	valid_1's auc: 0.846591	valid_1's binary_logloss: 0.452483
Early stopping, best iteration is:
[21]	training's auc: 0.86191	training's binary_logloss: 0.526094	valid_1's auc: 0.848403	valid_1's binary_logloss: 0.531382


In [20]:
X = df_1[final_model_var]
y = df_1[target]

from sklearn.feature_selection import SelectFromModel
train_x, valid_x, train_y, valid_y = train_test_split(X[final_model_var], y, test_size=0.1, shuffle=True, stratify=y, random_state=9001)


from xgboost import XGBClassifier
xgbc=XGBClassifier(n_estimators=10000, 
                   learning_rate=0.01, 
                   num_leaves=50, 
                   colsample_bytree=1,
                   reg_alpha=0.2, 
                   reg_lambda=50, 
                   min_split_gain=0.5, 
                   min_child_weight=30)

xgbc.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)
xgb_importance_df = pd.DataFrame()
xgb_importance_df["feature"] = final_model_var
xgb_importance_df["importance"] = xgbc.feature_importances_*(100)

xgbc.best_iteration

xgb_importance_df.to_excel('fi_xgb_oot.xlsx',index=False)


sub_preds = np.zeros(df.shape[0])
sub_preds = xgbc.predict_proba(df_1[final_model_var], xgbc.best_iteration)[:, 1] 
df['score'] = sub_preds

[0]	validation_0-auc:0.824742	validation_1-auc:0.792201
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.851241	validation_1-auc:0.809697
[200]	validation_0-auc:0.859552	validation_1-auc:0.822285
[300]	validation_0-auc:0.865205	validation_1-auc:0.824241
[400]	validation_0-auc:0.868373	validation_1-auc:0.828153
[500]	validation_0-auc:0.869817	validation_1-auc:0.832899
[600]	validation_0-auc:0.871005	validation_1-auc:0.835289
[700]	validation_0-auc:0.871616	validation_1-auc:0.837101
[800]	validation_0-auc:0.872063	validation_1-auc:0.838187
[900]	validation_0-auc:0.872633	validation_1-auc:0.839165
[1000]	validation_0-auc:0.872906	validation_1-auc:0.840107
[1100]	validation_0-auc:0.873033	validation_1-auc:0.840252
[1200]	validation_0-auc:0.873087	validation_1-auc:0.841411
[1300]	validation_0-auc:0.873353	validation_1-auc:0.842027
[1400]	validation_0-auc:0.87

In [21]:
#Model Results For Training Data
train_pred = xgbc.predict_proba(train_x[final_model_var], xgbc.best_iteration)[:, 1] 
train_pred = np.where(train_pred < 0.38, 0, 1)
confusion_matrix = pd.crosstab(train_y, train_pred, rownames=['Actual'], colnames=['Predicted'])
print('confusion_matrix: ',confusion_matrix)
from sklearn.metrics import roc_auc_score,precision_recall_fscore_support
print('roc_auc_score: ',roc_auc_score(train_y, train_pred))
precision, recall, f1_score, _ = precision_recall_fscore_support(train_y, train_pred, average='binary')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

confusion_matrix:  Predicted     0    1
Actual              
0          1575  346
1           259  901
roc_auc_score:  0.798304807122727
Precision:  0.7225340817963112
Recall:  0.7767241379310345
F1 score:  0.7486497714997923


In [22]:
#Model Results For Test Data
test_pred =  lgbc.predict_proba(valid_x[final_model_var],num_iteration=lgbc.best_iteration_)[:, 1] 
test_pred = np.where(test_pred < 0.38, 0, 1)
confusion_matrix = pd.crosstab(valid_y, test_pred, rownames=['Actual'], colnames=['Predicted'])

print('confusion_matrix: ',confusion_matrix)
from sklearn.metrics import roc_auc_score,precision_recall_fscore_support
print('roc_auc_score: ',roc_auc_score(valid_y, test_pred))
precision, recall, f1_score, _ = precision_recall_fscore_support(valid_y, test_pred, average='binary')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

confusion_matrix:  Predicted    0   1
Actual            
0          170  44
1           36  93
roc_auc_score:  0.7576613779613127
Precision:  0.6788321167883211
Recall:  0.7209302325581395
F1 score:  0.6992481203007519


In [23]:
#Model Results For Overall Data
oa_pred =  xgbc.predict_proba(df_1[final_model_var], xgbc.best_iteration)[:, 1] 
oa_pred = np.where(oa_pred < 0.38, 0, 1)
confusion_matrix = pd.crosstab(df_1[target], oa_pred, rownames=['Actual'], colnames=['Predicted'])

print('confusion_matrix: ',confusion_matrix)
from sklearn.metrics import roc_auc_score,precision_recall_fscore_support
print('roc_auc_score: ',roc_auc_score(df_1[target], oa_pred))
precision, recall, f1_score, _ = precision_recall_fscore_support(df_1[target], oa_pred, average='binary')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

confusion_matrix:  Predicted     0    1
Actual              
0          1745  390
1           296  993
roc_auc_score:  0.7938474172560833
Precision:  0.7180043383947939
Recall:  0.7703646237393328
F1 score:  0.7432634730538923


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

from sklearn.metrics import confusion_matrix, brier_score_loss, accuracy_score

def show_all(df, nrow=None, ncol=None):
    with pd.option_context('display.max_rows', nrow, 'display.max_columns', ncol):
        display(df)
        
def plot_roc(actual, predicted, fig_name='roc_curve'):
	fpr, tpr, threshold = metrics.roc_curve(actual, predicted)
	roc_auc = metrics.auc(fpr, tpr)
	plt.clf()
	plt.plot(fpr,tpr,color='red',label='AUC=%0.2f'%roc_auc)
	plt.plot([0,1],[0,1],'k')
	plt.xlim([0.0,1.0])
	plt.ylim([0.0,1.0])
	plt.grid()
	plt.xlabel('False Positive Rate')
	plt.ylabel('True Positive Rate')
	titleStr='ROC Curve'
	plt.title(titleStr)
	plt.legend(loc="lower right")
	plt.savefig(fig_name)
    
def get_decile_table(df, dv='dv', score='score'):
	df = df[[dv, score]].copy()
	df.sort_values(score, ascending=False, inplace = True)
	df['Decile'] = pd.qcut(np.arange(1, df.shape[0]+1, 1), 10, labels=False)
	df['Decile'] = df['Decile'] + 1
	brier_losses = df.groupby(['Decile']).apply(lambda x: brier_score_loss(x[dv], x[score]))
	
	decl_tbl = pd.crosstab(df['Decile'], df[dv])
	
	if 0 not in decl_tbl.columns.tolist():
		decl_tbl[0] = 0
		decl_tbl = decl_tbl[[0, 1]]
	elif 1 not in decl_tbl.columns.tolist():
		decl_tbl[1] = 0

	decl_tbl.columns = ['Non_Event', 'Event']
	decl_tbl.reset_index(inplace=True)
	decl_tbl = decl_tbl[['Decile', 'Event', 'Non_Event']]
	decl_tbl['Total'] = decl_tbl['Event'] + decl_tbl['Non_Event']
	decl_tbl['Cum_Event'] = np.cumsum(decl_tbl['Event'])
	decl_tbl['Cum_Non_Event'] = np.cumsum(decl_tbl['Non_Event'])
	decl_tbl['Cum_Total'] = np.cumsum(decl_tbl['Total'])
	decl_tbl['Cum_Event_Prop'] = decl_tbl['Cum_Event']/decl_tbl['Cum_Event'].max()
	decl_tbl['Cum_Non_Event_Prop'] = decl_tbl['Cum_Non_Event']/decl_tbl['Cum_Non_Event'].max()
	decl_tbl['Cum_Lift_Event'] = decl_tbl['Cum_Event_Prop']/(decl_tbl['Decile']/10)
	decl_tbl['KS'] = decl_tbl['Cum_Event_Prop'] - decl_tbl['Cum_Non_Event_Prop']
	decl_tbl['Brier_Score_Loss'] = brier_losses
	return decl_tbl


def get_decile_lift_curve(decl_tbl=None, df=None, dv='dv', score='score'):
	if decl_tbl is None:
		decl_tbl = get_decile_table(df, dv, score)
		
	return decl_tbl['Decile'].values, decl_tbl['Cum_Lift_Event'].values    



def plot_decile_lift_curve(decl_tbl=None, df=None, dv='dv', score='score', fig_name="decile_lift"):
	deciles, lifts = get_decile_lift_curve(decl_tbl=decl_tbl, df=df, dv=dv, score=score)
	plt.clf()
	plt.plot(deciles, lifts)
	plt.xlabel("Deciles")
	plt.ylabel("Cum Lift of an Event")
	plt.title("Decile Lift")
	plt.savefig(fig_name)
    
def get_confusion_matrix(df, prob_thresh = 0.5, labels=[0, 1], dv='dv', score='score'):
	con_mat = confusion_matrix(df[dv], df[score] > prob_thresh, labels=labels)
	return con_mat


plot_roc(df_1[target], oa_pred, 'roc')


oa_pred_1=xgbc.predict_proba(df_1[final_model_var], xgbc.best_iteration)[:, 1]
df_1['score']=oa_pred_1
x = get_confusion_matrix(df_1, prob_thresh =0.38, labels=[0, 1],dv=target, score='score')
print(x)
print(x.sum(axis=1), x.sum(axis=0), x.sum())


decl_table=get_decile_table(df_1, dv=target, score='score')

decl_table.to_csv('decl_table.csv')


[[1745  390]
 [ 296  993]]
[2135 1289] [2041 1383] 3424


In [25]:
decl_table

,Decile,Event,Non_Event,Total,Cum_Event,Cum_Non_Event,Cum_Total,Cum_Event_Prop,Cum_Non_Event_Prop,Cum_Lift_Event,KS,Brier_Score_Loss
0,1,329,14,343,329,14,343,0.255237,0.006557,2.552366,0.248679,NaN
1,2,275,67,342,604,81,685,0.468580,0.037939,2.342901,0.430641,0.040240
2,3,230,112,342,834,193,1027,0.647013,0.090398,2.156711,0.556615,0.156417
3,4,156,187,343,990,380,1370,0.768037,0.177986,1.920093,0.590051,0.215652
4,5,108,234,342,1098,614,1712,0.851823,0.287588,1.703646,0.564235,0.247822
5,6,68,274,342,1166,888,2054,0.904577,0.415925,1.507629,0.488652,0.216077
6,7,48,295,343,1214,1183,2397,0.941815,0.554098,1.345451,0.387717,0.159644
7,8,35,307,342,1249,1490,2739,0.968968,0.697892,1.211210,0.271076,0.120629
8,9,23,319,342,1272,1809,3081,0.986811,0.847307,1.096457,0.139505,0.091606
9,10,17,326,343,1289,2135,3424,1.000000,1.000000,1.000000,0.000000,0.062102


In [26]:
plot_decile_lift_curve(df=df_1, dv=target, score='score', fig_name="decile_lift")

In [27]:
# all_df['pre1'] = [1 if (value >= 0.375)  else 0 for value in all_df['prediction']]
# x=all_df.groupby([ 'cat'])[[dv, 'score_0_1']].agg({dv: ['count', 'mean'], 'score_0_1': ['mean']})
# x.to_csv('cat.csv')


# # num

# #age
# all_df['age'].fillna(0,inplace=True)
# t = pd.qcut(all_df['age'], q=10, labels=None, retbins=True, duplicates= 'drop')
# all_df['age'] = t[0]
# x = all_df.groupby(['age'])[[dv, 'score_0_1']].mean().reset_index()
# x.to_csv('age.csv')